In [1]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ================================

## Two-Stage Recommender Systems

In large scale recommender systems pipelines, the size of the item catalog (number of unique items) might be in the order of millions. At such scale, a typical setup is having two-stage pipeline, where a faster candidate retrieval model quickly extracts thousands of relevant items and a then a more powerful ranking model (i.e. with more features and more powerful architecture) ranks the top-k items that are going to be displayed to the user. For ML-based candidate retrieval model, as it needs to quickly score millions of items for a given user, a popular choices are models that can produce recommendation scores by just computing the dot product the user embeddings and item embeddings. Popular choices of such models are **Matrix Factorization**, which learns low-rank user and item embeddings, and the **Two-Tower architecture**, which is a neural network with two MLP towers where both user and item features are fed to generate user and item embeddings in the output.


### Dataset

In this notebook, we are building a Two-Tower model for Item Retrieval task using the Ali-CCP: Alibaba Click and Conversion Prediction dataset. To download the Ali-CCP training and test datasets visit [tianchi.aliyun.com](https://tianchi.aliyun.com/dataset/dataDetail?dataId=408#1) website. We have curated the raw dataset using this [script]() and generated the parquet files that we will use for this example.

### Learning objectives
- Preparing the data with NVTabular
- Training and evaluating Two-Tower model with Merlin Models
- Exporting the model for deployment

### Feature Engineering with NVTabular

In [2]:
import os

os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"
import cudf
import glob
import gc

import nvtabular as nvt
from nvtabular.ops import *
from merlin.models.utils.example_utils import workflow_fit_transform

from merlin.schema.tags import Tags
from merlin.schema import Schema

import merlin.models.tf as mm
import merlin.models.tf.dataset as tf_dataloader

from merlin.io.dataset import Dataset
from merlin.schema.io.tensorflow_metadata import TensorflowMetadata
from merlin.models.tf.blocks.core.aggregation import CosineSimilarity

import tensorflow as tf

2022-03-25 18:21:09.740734: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-25 18:21:10.876420: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:214] Using CUDA malloc Async allocator for GPU: 0
2022-03-25 18:21:10.876553: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 16254 MB memory:  -> device: 0, name: Quadro GV100, pci bus id: 0000:15:00.0, compute capability: 7.0


In [3]:
# disable INFO and DEBUG logging everywhere
import logging

logging.disable(logging.WARNING)

First, we define our input and output paths. We will use the parquet files in the test folder to validate our trained model.

In [4]:
DATA_FOLDER = os.environ.get("DATA_FOLDER", "/workspace/data/")
train_path = os.path.join(DATA_FOLDER, "train", "*.parquet")
valid_path = os.path.join(DATA_FOLDER, "test", "*.parquet")
output_path = "/workspace/data/processed/"

ETL Workflow

We keep only positive interactions where clicks==1 in the dataset with `Filter()` op.

In [5]:
user_id = ["user_id"] >> Categorify() >> TagAsUserID()
item_id = ["item_id"] >> Categorify() >> TagAsItemID()

item_features = ["item_category", "item_shop", "item_brand"] >> Categorify() >> TagAsItemFeatures()

user_features = (
    [
        "user_shops",
        "user_profile",
        "user_group",
        "user_gender",
        "user_age",
        "user_consumption_2",
        "user_is_occupied",
        "user_geography",
        "user_intentions",
        "user_brands",
        "user_categories",
    ]
    >> Categorify()
    >> TagAsUserFeatures()
)

inputs = user_id + item_id + item_features + user_features + ["click"]

outputs = inputs >> Filter(f=lambda df: df["click"] == 1)

workflow_fit_transform(outputs, train_path, valid_path, output_path)

/usr/local/lib/python3.8/dist-packages/cudf/core/dataframe.py:1253: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(


## Building a Two-Tower Model with Merlin Models

We will use Two-Tower Model for item retrieval task. Real-world large scale recommender systems have hundreds of millions of items (products) and users. Thus, these systems often composed of two stages: candidate generation (retrieval) and ranking (scoring the retrieved items). At candidate generation step, a subset of relevant items from large item corpus is retrieved. You can read more about two stage Recommender Systems here. In this example, we're going to focus on the retrieval stage.

A Two-Tower Model consists of item (candidate) and user (query) encoder towers. With two towers, the model can learn representations (embeddings) for queries and candidates separately. 

<img src="./images/TwoTower.png"  width="30%">

Image Adapted from: [Off-policy Learning in Two-stage Recommender Systems](https://dl.acm.org/doi/abs/10.1145/3366423.3380130)

We use the `schema` object to define our model.

In [6]:
train = Dataset(os.path.join(output_path, "train", "*.parquet"), part_size="500MB")
valid = Dataset(os.path.join(output_path, "valid", "*.parquet"), part_size="500MB")

schema = train.schema

In [7]:
schema = schema.select_by_tag([Tags.ITEM_ID, Tags.USER_ID, Tags.ITEM, Tags.USER])

We can print out the feature column names.

In [8]:
schema.column_names

['user_id',
 'item_id',
 'item_category',
 'item_shop',
 'item_brand',
 'user_shops',
 'user_profile',
 'user_group',
 'user_gender',
 'user_age',
 'user_consumption_2',
 'user_is_occupied',
 'user_geography',
 'user_intentions',
 'user_brands',
 'user_categories']

We expect the label names to be empty.

In [9]:
label_names = schema.select_by_tag(Tags.TARGET).column_names
label_names

[]

### Negative sampling
Many datasets for recommender systems contains implicit feedback, with logs of user interactions like clicks, add-to-cart, purchases, music listening events, rather than explicit ratings that reflects user preferences over items. To be able to learn from implicit feedback, we use the general (and naive) assumption that the interacted items are more relevant for the user than the non-interacted ones.
In Merlin Models we provide some scalable negative sampling algorithms for the Item Retrieval Task. In particular, we use in this example the in-batch sampling algorithm which uses the items interacted by other users as negatives within the same mini-batch.

### Building the Model

Now, let's build our Two-Tower model. In a nutshell, we aggregate all user features to feed in user tower and feed the item features to the item tower. Then we compute the positive score by multiplying the user embedding with the item embedding and sample negative items (read more about negative sampling [here](https://openreview.net/pdf?id=824xC-SgWgU) and [here](https://medium.com/mlearning-ai/overview-negative-sampling-on-recommendation-systems-230a051c6cd7)), whose item embeddings are also multiplied by the user embedding. Then we apply the loss function on top of the positive and negative scores.

In [10]:
model = mm.TwoTowerModel(
    schema,
    query_tower=mm.MLPBlock([128, 64], no_activation_last_layer=True),
    loss="categorical_crossentropy",
    samplers=[mm.InBatchSampler()],
    embedding_options=mm.EmbeddingOptions(infer_embedding_sizes=True),
    metrics=[mm.RecallAt(10), mm.NDCGAt(10)],
)

Let's explain the parameters in the TwoTowerModel():
- no_activation_last_layer: when set True, no activation is used for top hidden layer. Learn more [here](https://storage.googleapis.com/pub-tools-public-publication-data/pdf/b9f4e78a8830fe5afcf2f0452862fb3c0d6584ea.pdf).
- infer_embedding_sizes: when set True, automatically defines the embedding dimension from the feature cardinality in the schema

**Metrics:**

The following information retrieval metrics are used to compute the Top-10 accuracy of recommendation lists containing all items:

- **Normalized Discounted Cumulative Gain (NDCG@10)**: NDCG accounts for rank of the relevant item in the recommendation list and is a more fine-grained metric than HR, which only verifies whether the relevant item is among the top-k items.

- **Recall@10**: Also known as HitRate@n when there is only one relevant item in the recommendation list. Recall just verifies whether the relevant item is among the top-n items.

We need to initialize the dataloaders.

In [11]:
model.set_retrieval_candidates_for_evaluation(train)

opt = tf.keras.optimizers.Adagrad(learning_rate=0.003)
model.compile(optimizer=opt, run_eagerly=False)
model.fit(train, validation_data=valid, batch_size=4096, epochs=5)

2022-03-25 18:21:49.247382: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Epoch 1/5
382/384 [============================>.] - ETA: 0s - recall_at_10: 0.0024 - ndcg_10: 0.0011 - loss: 8.3225 - regularization_loss: 0.0000e+00 - total_loss: 8.3225

2022-03-25 18:22:08.000481: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1034311152 exceeds 10% of free system memory.
2022-03-25 18:22:11.491670: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1034311152 exceeds 10% of free system memory.
2022-03-25 18:22:21.411679: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: cond/then/_0/cond/cond/branch_executed/_184


384/384 [==============================] - 37s 80ms/step - recall_at_10: 0.0024 - ndcg_10: 0.0011 - loss: 8.3196 - regularization_loss: 0.0000e+00 - total_loss: 8.3196 - val_recall_at_10: 0.0025 - val_ndcg_10: 0.0011 - val_loss: 7.4208 - val_regularization_loss: 0.0000e+00 - val_total_loss: 7.4208
Epoch 2/5
382/384 [============================>.] - ETA: 0s - recall_at_10: 0.0024 - ndcg_10: 0.0011 - loss: 8.3217 - regularization_loss: 0.0000e+00 - total_loss: 8.3217

2022-03-25 18:22:38.212795: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1034311152 exceeds 10% of free system memory.
2022-03-25 18:22:41.619888: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1034311152 exceeds 10% of free system memory.


384/384 [==============================] - 28s 71ms/step - recall_at_10: 0.0024 - ndcg_10: 0.0011 - loss: 8.3188 - regularization_loss: 0.0000e+00 - total_loss: 8.3188 - val_recall_at_10: 0.0025 - val_ndcg_10: 0.0012 - val_loss: 7.4200 - val_regularization_loss: 0.0000e+00 - val_total_loss: 7.4200
Epoch 3/5
382/384 [============================>.] - ETA: 0s - recall_at_10: 0.0024 - ndcg_10: 0.0011 - loss: 8.3210 - regularization_loss: 0.0000e+00 - total_loss: 8.3210

2022-03-25 18:23:05.813015: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1034311152 exceeds 10% of free system memory.


384/384 [==============================] - 28s 72ms/step - recall_at_10: 0.0024 - ndcg_10: 0.0011 - loss: 8.3181 - regularization_loss: 0.0000e+00 - total_loss: 8.3181 - val_recall_at_10: 0.0025 - val_ndcg_10: 0.0012 - val_loss: 7.4193 - val_regularization_loss: 0.0000e+00 - val_total_loss: 7.4193
Epoch 4/5
384/384 [==============================] - 28s 72ms/step - recall_at_10: 0.0025 - ndcg_10: 0.0011 - loss: 8.3175 - regularization_loss: 0.0000e+00 - total_loss: 8.3175 - val_recall_at_10: 0.0026 - val_ndcg_10: 0.0012 - val_loss: 7.4187 - val_regularization_loss: 0.0000e+00 - val_total_loss: 7.4187
Epoch 5/5
384/384 [==============================] - 28s 72ms/step - recall_at_10: 0.0025 - ndcg_10: 0.0011 - loss: 8.3169 - regularization_loss: 0.0000e+00 - total_loss: 8.3169 - val_recall_at_10: 0.0026 - val_ndcg_10: 0.0012 - val_loss: 7.4182 - val_regularization_loss: 0.0000e+00 - val_total_loss: 7.4182


In batch sampling is prone to popularity-bias. since different users might have interacted same items. # in the fit method we are only using the negative sampling in each batch and the final score is avg over all the batches. 
for each batch we aer comparing only the items in the given batch. in the `.evaluate()` we are considering entire item catalog for each positive item. generate the scores for all items and check the position of the positive item in the list of scores.

In [12]:
model.evaluate(valid, return_dict=True, batch_size=1024)

1542/1542 [==============================] - 46s 29ms/step - recall_at_10: 2.2805e-05 - ndcg_10: 9.9212e-06 - loss: 5.2483e-05 - regularization_loss: 0.0000e+00 - total_loss: 5.2483e-05


{'recall_at_10': 2.280459739267826e-05,
 'ndcg_10': 9.921227501763497e-06,
 'loss': 0.0,
 'regularization_loss': 0.0,
 'total_loss': 0.0}

## Exporting Retrieval Models

So far we have trained and evaluated our Retrieval model. Now, the next step is to deploy our model and generate top-K recommendations given a user (query). We can efficiently serve our model by indexing the trained item embeddings into an **Approximate Nearest Neighbors (ANN)** engine. Basically, for a given user query vector, that is generated passing the user features into user tower of retrieval model, we do an ANN search query to find the ids of nearby item vectors, and at serve time, we score user embeddings over all indexed top-K item embeddings within the ANN engine.

In doing so, we need to export
 
- user (query) tower
- item and user features
- item embeddings

#### Save User (query) tower

We are able to save the user tower model as a TF model to disk. The user tower model is needed to generate a user embedding vector when a user feature vector <i>x</i> is fed into that model.

In [13]:
query_tower = model.retrieval_block.query_block()
query_tower.save("query_tower")

#### Extract and save User features

With `unique_rows_by_features` utility function we can easily extract both unique user and item features tables as cuDF dataframes. Note that for user features table, we use `USER` and `USER_ID` tags.

In [14]:
from merlin.models.utils.dataset import unique_rows_by_features

user_features = (
    unique_rows_by_features(train, Tags.USER, Tags.USER_ID).compute().reset_index(drop=True)
)

In [15]:
user_features.head()

user_id  user_shops  user_profile  user_group  user_gender  user_age  \
0        0           0             1           4            1         3   
1        1         109             0           0            0         0   
2        2         301             1           1            1         1   
3        3        1876            23           7            2         3   
4        4         534             1           2            1         2   

   user_consumption_2  user_is_occupied  user_geography  user_intentions  \
0                   2                 1               0                0   
1                   0                 0               0               69   
2                   1                 1               2               57   
3                   1                 1               1                5   
4                   1                 1               0               40   

   user_brands  user_categories  
0            0                0  
1          131                9  
2         4709               57  
3           63                3  
4           22              108

In [16]:
user_features.shape

(214994, 12)

In [17]:
# save to disk
user_features.to_parquet("user_features.parquet")

#### Extract and save Item features

In [18]:
item_features = (
    unique_rows_by_features(train, Tags.ITEM, Tags.ITEM_ID).compute().reset_index(drop=True)
)

In [19]:
item_features.head()

item_id  item_category  item_shop  item_brand
0        0              0          0           0
1        1            441        432         474
2        2            193       1159         125
3        3              3       1463         872
4        4            282       2479         555

In [20]:
# save to disk
item_features.to_parquet("item_features.parquet")

#### Extract and save Item embeddings

In [21]:
item_embs = model.item_embeddings(Dataset(item_features, schema=schema), batch_size=1024)
item_embs_df = item_embs.compute(scheduler="synchronous")

In [22]:
item_embs_df

item_id  item_category  item_shop  item_brand         0         1  \
0             0              0          0           0  0.100480 -0.101183   
1             1            441        432         474  0.106592 -0.140783   
2             2            193       1159         125 -0.028713 -0.173785   
3             3              3       1463         872  0.087634 -0.136825   
4             4            282       2479         555  0.044352 -0.185963   
...         ...            ...        ...         ...       ...       ...   
568842  3078159           1764     365864      121053 -0.181527  0.013411   
568843  3078210            377      97495      116656 -0.000746 -0.059242   
568844  3078270           3817     577371       80796 -0.073659 -0.020232   
568845  3078283            551     201089       85172 -0.013963 -0.047382   
568846  3078304            475     255840         335 -0.191559 -0.092710   

               2         3         4         5  ...        54        55  \
0       0.038712 -0.069887  0.055463  0.187399  ... -0.136141  0.092087   
1       0.187643  0.116384 -0.117863  0.151208  ... -0.078249  0.098082   
2      -0.056876  0.034120 -0.056204  0.311673  ... -0.129029  0.144884   
3       0.128481  0.016375  0.066299 -0.077430  ... -0.183097  0.093180   
4      -0.060681 -0.028867  0.046313  0.220343  ... -0.085646  0.063820   
...          ...       ...       ...       ...  ...       ...       ...   
568842  0.059540  0.030576  0.007284  0.240745  ... -0.156806  0.170368   
568843  0.196628  0.018706  0.065176  0.227332  ... -0.049041  0.105531   
568844 -0.099983 -0.005017 -0.238924  0.178821  ... -0.213560  0.130538   
568845  0.124333  0.055950  0.104881  0.094359  ...  0.186676  0.136704   
568846  0.025378 -0.024475  0.079726  0.079302  ...  0.044710  0.082819   

              56        57        58        59        60        61        62  \
0       0.000309  0.012232  0.004694  0.185416 -0.175290 -0.113102  0.046894   
1       0.033912  0.028970  0.004288  0.182896 -0.049048  0.088471  0.095434   
2      -0.012147  0.056141 -0.024954  0.147257 -0.034835  0.065109  0.052829   
3       0.074107 -0.220991  0.077473  0.183373 -0.085114 -0.160506 -0.069322   
4      -0.137686 -0.071112 -0.161351  0.150922 -0.066201 -0.178179  0.023789   
...          ...       ...       ...       ...       ...       ...       ...   
568842  0.061671 -0.153007 -0.216809  0.114002  0.040904 -0.152487  0.138781   
568843 -0.003052  0.104010 -0.052346 -0.033157 -0.106754 -0.065986  0.317890   
568844  0.054929 -0.038743 -0.068914  0.062025  0.089917  0.031116  0.102875   
568845  0.101370 -0.016214 -0.030931  0.129269 -0.069209 -0.014182  0.144155   
568846  0.032549 -0.024786 -0.024001  0.074184 -0.074518 -0.077085  0.170670   

              63  
0      -0.095183  
1      -0.031751  
2      -0.073503  
3       0.011579  
4      -0.196832  
...          ...  
568842  0.004465  
568843 -0.063438  
568844 -0.180703  
568845 -0.062910  
568846  0.116576  

[568847 rows x 68 columns]

In [23]:
# select only embedding columns
item_embeddings = item_embs_df.iloc[:, 4:]

In [24]:
item_embeddings.head()

0         1         2         3         4         5         6  \
0  0.100480 -0.101183  0.038712 -0.069887  0.055463  0.187399  0.088853   
1  0.106592 -0.140783  0.187643  0.116384 -0.117863  0.151208  0.165833   
2 -0.028713 -0.173785 -0.056876  0.034120 -0.056204  0.311673  0.211417   
3  0.087634 -0.136825  0.128481  0.016375  0.066299 -0.077430  0.176676   
4  0.044352 -0.185963 -0.060681 -0.028867  0.046313  0.220343  0.015842   

          7         8         9  ...        54        55        56        57  \
0 -0.251354  0.069880 -0.046955  ... -0.136141  0.092087  0.000309  0.012232   
1 -0.152995 -0.071885  0.057338  ... -0.078249  0.098082  0.033912  0.028970   
2 -0.058011  0.129465 -0.073758  ... -0.129029  0.144884 -0.012147  0.056141   
3 -0.230770  0.042750 -0.019357  ... -0.183097  0.093180  0.074107 -0.220991   
4 -0.183577  0.102639 -0.070050  ... -0.085646  0.063820 -0.137686 -0.071112   

         58        59        60        61        62        63  
0  0.004694  0.185416 -0.175290 -0.113102  0.046894 -0.095183  
1  0.004288  0.182896 -0.049048  0.088471  0.095434 -0.031751  
2 -0.024954  0.147257 -0.034835  0.065109  0.052829 -0.073503  
3  0.077473  0.183373 -0.085114 -0.160506 -0.069322  0.011579  
4 -0.161351  0.150922 -0.066201 -0.178179  0.023789 -0.196832  

[5 rows x 64 columns]

In [25]:
# save to disk
item_embeddings.to_parquet("item_embeddings.parquet")

That's it. You have learned how to train and evaluate your Two-Tower retrieval model, and then how to export the required components to be able to deploy this model to generate recommendations. In order to learn more on serving a model to [Triton Inference Server](https://github.com/triton-inference-server/server), please explore the examples in the [Merlin](https://github.com/NVIDIA-Merlin/Merlin) and [Merlin Systems](https://github.com/NVIDIA-Merlin/systems) repos.